In [1]:
from dependencies import *
import torch

In [3]:
checkpoint_quick_test= torch.load("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/checkpoints/model_checkpoint_UNet_01.pth")

In [5]:
#Checking the parameters and keys
print(type(checkpoint_quick_test))

<class 'dict'>


In [6]:
print(checkpoint_quick_test.keys())

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'loss'])
